In [1]:
# NOTE: REQUIRES PROJ>=9.2 and GEOPANDAS>=0.13

#!pip install geopandas==0.13

In [2]:
import os
os.environ['USE_PYGEOS'] = '0'
os.environ['PROJ_DEBUG'] = '2' # For showing logs
#os.environ['PROJ_NETWORK'] = 'ON' # Ensure this is 'ON' to get shift grids over the internet
print(os.environ['PROJ_NETWORK']) 

import fiona
import geopandas as gpd
import shapely

ON


In [3]:
gpd.show_versions()


SYSTEM INFO
-----------
python     : 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
executable : /srv/conda/envs/notebook/bin/python
machine    : Linux-5.10.167-147.601.amzn2.x86_64-x86_64-with-glibc2.27

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : 3.11.2
GEOS lib   : None
GDAL       : 3.7.0
GDAL data dir: /srv/conda/envs/notebook/share/gdal
PROJ       : 9.2.1
PROJ data dir: /srv/conda/envs/notebook/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.13.0
numpy      : 1.23.5
pandas     : 1.5.1
pyproj     : 3.6.0
shapely    : 2.0.1
fiona      : 1.9.4
geoalchemy2: None
geopy      : None
matplotlib : 3.6.2
mapclassify: 2.5.0
pygeos     : 0.14
pyogrio    : None
psycopg2   : 2.9.6 (dt dec pq3 ext lo64)
pyarrow    : 12.0.1
rtree      : 1.0.1


In [4]:
# from slidrule single granule, default parameters
#gf = icesat2.atl06p({}, resources=['ATL03_20181019224323_03250112_005_01.h5'])
gf = gpd.read_file('sliderule-output.gpkg',
                  rows=100) 

In [5]:
gf.geometry.head(2)

0    POINT (-100.63246 -78.98922)
1    POINT (-100.63265 -78.98904)
Name: geometry, dtype: geometry

In [6]:
# Always check wehere you data is!
points = gf.reset_index()
points.loc[:, 'time'] = points.time.dt.strftime('%Y-%m-%d')
points.explore()

/tmp/ipykernel_2695/4052444587.py:3: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  points.loc[:, 'time'] = points.time.dt.strftime('%Y-%m-%d')


In [7]:
# Get bounding box of all of our points
w,s,e,n = gf.unary_union.bounds #W, S, E, N
print(w,s,e,n)

-100.94678306984892 -79.00104942913823 -100.63246339163007 -78.98240211911293


In [8]:
#!projinfo -s EPSG:7912 -t EPSG:9518 -o PROJ --hide-ballpark --spatial-test intersects
!projinfo -s EPSG:7912 -t EPSG:9518 -o PROJ --bbox {w},{s},{e},{n}  --hide-ballpark --spatial-test intersects

pj_open_lib(proj.ini): call fopen(/srv/conda/envs/notebook/share/proj/proj.ini) - succeeded
pj_open_lib(proj.db): call fopen(/srv/conda/envs/notebook/share/proj/proj.db) - succeeded
pj_open_lib(us_noaa_nadcon5_nad27_nad83_1986_conus.tif): call fopen(/srv/conda/envs/notebook/share/proj/us_noaa_nadcon5_nad27_nad83_1986_conus.tif) - failed
pj_open_lib(proj.db): call fopen(/srv/conda/envs/notebook/share/proj/proj.db) - succeeded
pj_open_lib(us_noaa_nadcon5_nad83_1986_nad83_harn_conus.tif): call fopen(/srv/conda/envs/notebook/share/proj/us_noaa_nadcon5_nad83_1986_nad83_harn_conus.tif) - failed
pj_open_lib(us_noaa_nadcon5_nad83_harn_nad83_fbn_conus.tif): call fopen(/srv/conda/envs/notebook/share/proj/us_noaa_nadcon5_nad83_harn_nad83_fbn_conus.tif) - failed
pj_open_lib(us_noaa_nadcon5_nad83_fbn_nad83_2007_conus.tif): call fopen(/srv/conda/envs/notebook/share/proj/us_noaa_nadcon5_nad83_fbn_nad83_2007_conus.tif) - failed
pj_open_lib(us_noaa_nadcon5_nad83_2007_nad83_2011_conus.tif): call fopen(/

In [9]:
# NOTE: only vshift so only expect vertical coordinate change
points3D = gpd.points_from_xy(gf.geometry.x, gf.geometry.y, gf.h_mean)
gf3D = gpd.GeoDataFrame(geometry=points3D, crs='EPSG:7912')
gf3D.head(2)

,geometry
0,POINT Z (-100.63246 -78.98922 1875.21728)
1,POINT Z (-100.63265 -78.98904 1875.36690)


In [10]:
gf3D.crs

<Geographic 3D CRS: EPSG:7912>
Name: ITRF2014
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
- h[up]: Ellipsoidal height (metre)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: International Terrestrial Reference Frame 2014
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [11]:
shapely.get_coordinates(gf3D.iloc[0], include_z=True)

array([[-100.63246339,  -78.98921541, 1875.21727596]])

In [12]:
gfGeoid = gf3D.to_crs(epsg=9518)
shapely.get_coordinates(gfGeoid.iloc[0], include_z=True)

array([[-100.63246339,  -78.98921541, 1904.67380708]])

In [13]:
#gfGeoid.head()

In [14]:
# SHow debug log

# CAREFUL, geopandas will happily do bogus transforms for you

import logging
logging.basicConfig(level=logging.DEBUG)

with fiona.Env(CPL_DEBUG=True):
    test = gf3D.to_crs(epsg="2927+5703")

DEBUG:fiona._env:GDAL_DATA found in environment.
DEBUG:fiona._env:PROJ_DATA found in environment.
DEBUG:pyproj:PROJ_DEBUG: pj_open_lib(us_noaa_nadcon5_nad83_harn_nad83_fbn_conus.tif): call fopen(/home/jovyan/.local/share/proj/us_noaa_nadcon5_nad83_harn_nad83_fbn_conus.tif) - failed
DEBUG:pyproj:PROJ_DEBUG: pj_open_lib(us_noaa_nadcon5_nad83_fbn_nad83_2007_conus.tif): call fopen(/home/jovyan/.local/share/proj/us_noaa_nadcon5_nad83_fbn_nad83_2007_conus.tif) - failed
DEBUG:pyproj:PROJ_DEBUG: pj_open_lib(us_noaa_nadcon5_nad83_2007_nad83_2011_conus.tif): call fopen(/home/jovyan/.local/share/proj/us_noaa_nadcon5_nad83_2007_nad83_2011_conus.tif) - failed
DEBUG:pyproj:PROJ_DEBUG: pj_open_lib(us_noaa_g2018u0.tif): call fopen(/home/jovyan/.local/share/proj/us_noaa_g2018u0.tif) - failed
DEBUG:pyproj:PROJ_DEBUG: pj_open_lib(g2018u0.gtx): call fopen(/home/jovyan/.local/share/proj/g2018u0.gtx) - failed
DEBUG:pyproj:PROJ_DEBUG: pj_open_lib(us_noaa_geoid03_conus.tif): call fopen(/home/jovyan/.local/sha

In [15]:
test.head()

,geometry
0,POINT Z (53991548.388 -183138438.398 1875.217)
1,POINT Z (53990465.763 -183136201.989 1875.367)
2,POINT Z (53989380.806 -183133966.815 1875.252)
3,POINT Z (53988291.113 -183131734.019 1875.504)
4,POINT Z (53987200.796 -183129501.561 1875.526)
